# Deep Learning (Aprendizaje Profundo)
---
<style>
      h1, h2, h3, h4, h5, h6,.imagen {
        text-align: center;
      }
 img{width: 75%; height: 75%;}
</style>

- [Deep Learning (Aprendizaje Profundo)](#deep-learning-aprendizaje-profundo)
  - [Es o no es una pizza](#es-o-no-es-una-pizza)
  - [Neuronas](#neuronas)
    - [Estructura de una neurona](#estructura-de-una-neurona)
  - [Neuronas artificiales](#neuronas-artificiales)
    - [Estructura de una neurona artificial](#estructura-de-una-neurona-artificial)
      - [Funciones de activación](#funciones-de-activación)
        - [Función Sigmoide (Sigmoid)](#función-sigmoide-sigmoid)
        - [Función ReLU (Rectified Linear Unit)](#función-relu-rectified-linear-unit)
        - [Función Leaky ReLU](#función-leaky-relu)
        - [Función Tanh (Tangente hiperbólica)](#función-tanh-tangente-hiperbólica)
        - [Función Identidad (Linear)](#función-identidad-linear)
  - [Redes neuronales artificiales](#redes-neuronales-artificiales)
    - [Arquitecturas](#arquitecturas)
      - [Perceptrón](#perceptrón)
      - [Redes Neuronales Feedforward](#redes-neuronales-feedforward)
      - [Redes Neuronales Recurrentes (RNN)](#redes-neuronales-recurrentes-rnn)
      - [Redes neuronales convolucionales (ConvNets o CNN)](#redes-neuronales-convolucionales-convnets-o-cnn)
      - [Variational Autoencoder (VAE)](#variational-autoencoder-vae)
      - [UNET](#unet)
      - [Transformer](#transformer)
    - [Deep Learning](#deep-learning)
    - [Entrenamiento de una red neuronal](#entrenamiento-de-una-red-neuronal)
      - [pytorch](#pytorch)

## Es o no es una pizza

### Instalar librerías 

```jupyterpython
%pip install torch torchvision datasets matplotlib sklearn pandas tqdm

```


### Datos

```python
 
from datasets import load_dataset
from PIL import Image


# Cargamos el conjunto de datos "pizza_not_pizza" con la partición "train"
dataset = load_dataset("nateraw/pizza_not_pizza", split="train")

# Mezclamos aleatoriamente los datos en el conjunto de datos
dataset = dataset.shuffle()

# Convertimos el conjunto de datos a un objeto Pandas DataFrame
dataset = dataset.to_pandas()

# Mostramos las primeras filas del DataFrame
dataset.head()

# Seleccionamos aleatoriamente el 80% de los datos para el conjunto de entrenamiento
train = dataset.sample(frac=0.8)

# Eliminamos los datos seleccionados para el conjunto de entrenamiento del conjunto de datos original para obtener el conjunto de prueba
test = dataset.drop(train.index)

#visualizamos una imagen
Image.open(dataset['image'][4]['path']).resize((256,256))

```


### Transformaciones

```python
# Importamos la función transforms del módulo torchvision
from torchvision import transforms

# Definimos los valores de media, desviación estándar y tamaño de imagen
media = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
size = 192

# Definimos la transformación para el conjunto de entrenamiento
train_transform = transforms.Compose([
    transforms.Resize([size, size]),  # Redimensionamos la imagen a tamaño 192x192
    transforms.ToTensor(),  # Convertimos la imagen a un tensor
    transforms.Normalize(media, std)  # Normalizamos los valores de los píxeles de la imagen
])

# Definimos la transformación para el conjunto de prueba
test_transform = transforms.Compose([
    transforms.Resize([size, size]),  # Redimensionamos la imagen a tamaño 192x192
    transforms.ToTensor(),  # Convertimos la imagen a un tensor
    transforms.Normalize(media, std)  # Normalizamos los valores de los píxeles de la imagen
])

```


### Datasets

```python
# Importamos la clase Dataset del módulo torch.utils.data
from torch.utils.data import Dataset

class PizzaNotPizzaDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.values  # Convertimos el DataFrame a un arreglo de Numpy
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)  # Devuelve la longitud del arreglo
        
    def __getitem__(self, idx):
        image = Image.open(self.dataframe[idx][0]['path'])  # Abrimos la imagen utilizando la ruta de la imagen en el DataFrame
        label = self.dataframe[idx][1]  # Obtenemos la etiqueta de la imagen
        
        if self.transform:
            image = self.transform(image)  # Aplicamos la transformación a la imagen si se especificó
            
        return image, label  # Devolvemos la imagen y su etiqueta como una tupla

# Creamos un objeto de la clase PizzaNotPizzaDataset para el conjunto de entrenamiento
train_dataset = PizzaNotPizzaDataset(train, transform=train_transform)

# Creamos un objeto de la clase PizzaNotPizzaDataset para el conjunto de prueba
test_dataset = PizzaNotPizzaDataset(test, transform=test_transform)

# Obtenemos la primera imagen y su etiqueta del conjunto de entrenamiento
imagen, etiqueta = train_dataset[0]

# Mostramos la forma de la imagen y su etiqueta
imagen.shape, etiqueta

```

        

### Dataloaders

```python

from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# Definimos el tamaño del lote (batch size) como 32
bs = 32

# Creamos un objeto DataLoader para el conjunto de entrenamiento
train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

# Creamos un objeto DataLoader para el conjunto de prueba
test_dataloader = DataLoader(test_dataset, batch_size=bs, shuffle=False)


# Obtenemos un lote de datos de entrenamiento utilizando el objeto DataLoader
btest = iter(train_dataloader).next()

# Definimos la cantidad de imágenes que se mostrarán, así como el número de filas y columnas en la figura
cantidad = 20
filas = 4
columnas = 5

# Convertimos los valores de media y desviación estándar a tensores de PyTorch
tensormedia = torch.tensor(media)
tensorstd = torch.tensor(std)

# Creamos una figura con subfiguras utilizando la función subplots() de pyplot
fig, axs = plt.subplots(filas, columnas, figsize=(15, 10))

# Iteramos sobre las imágenes en el lote y las mostramos en las subfiguras
for i in range(cantidad):
    ax = axs[i//columnas, i%columnas]
    ax.imshow(btest[0][i].permute(1,2,0)*tensorstd+tensormedia )  # Mostramos la imagen con los valores de píxeles normalizados
    ax.axis('off')
    ax.set_title("pizza" if btest[1][i].item()==1 else "not pizza")  # Mostramos la etiqueta de la imagen como título de la subfigura

```


### Modelo

```python
# Importamos el módulo models del paquete torchvision y el módulo nn de PyTorch
from torchvision import models
from torch import nn

# Definimos la clase PizzaNotPizzaModel que hereda de la clase nn.Module
class PizzaNotPizzaModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Cargamos el modelo ResNet18 pre-entrenado en ImageNet y congelamos sus parámetros
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1, progress=True)
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Modificamos la capa completamente conectada (fc) del modelo para adaptarlo a nuestro problema
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),  # Añadimos una capa lineal con 512 neuronas
            nn.ReLU(inplace=True),  # Añadimos una función de activación ReLU
            nn.Dropout(0.2),  # Añadimos una capa de dropout con una probabilidad de 0.2
            nn.Linear(512, num_classes)  # Añadimos una capa lineal con el número de clases de nuestro problema
        )
        
    def forward(self, x):
        return self.model(x)  # Devolvemos la salida del modelo para una entrada x

# Creamos un objeto de la clase PizzaNotPizzaModel con 2 clases de salida
modelo = PizzaNotPizzaModel(2)

# Pasamos un lote de datos de entrenamiento al modelo y obtenemos la salida
modelo(btest[0])

```
        

### Hiperparámetros y metricas

```python
# Creamos un objeto de dispositivo que utiliza la GPU si está disponible, de lo contrario utiliza la CPU
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mostramos el dispositivo utilizado
print(dispositivo)

# Definimos la tasa de aprendizaje y el número de épocas
lr = 3e-3
epocas = 5

# Creamos un objeto de la clase PizzaNotPizzaModel y lo movemos al dispositivo especificado
modelo = PizzaNotPizzaModel(2)
modelo.to(dispositivo)

# Definimos la función de pérdida y el optimizador
funciondeperdida = nn.CrossEntropyLoss()
optimizador = torch.optim.SGD(modelo.parameters(), lr=lr, momentum=0.9)


# Definimos las listas para almacenar las métricas de entrenamiento y prueba
perdidas_train = []  # Lista para almacenar las pérdidas de entrenamiento
perdidas_test = []  # Lista para almacenar las pérdidas de prueba

error_train = []  # Lista para almacenar los errores de entrenamiento
error_test = []  # Lista para almacenar los errores de prueba

Acuracy_train = []  # Lista para almacenar las precisiones de entrenamiento
Acuracy_test = []  # Lista para almacenar las precisiones de prueba

```

 

### Bucle de entrenamiento

```python
# Iteramos sobre el número de épocas especificado
for epoca in (range(epocas)):
    # Ponemos el modelo en modo de entrenamiento
    modelo.train()
    
    # Inicializamos las variables de pérdida, precisión y error para el conjunto de entrenamiento
    perdida_train = 0
    acuracy_train = 0
    error  = 0
    
    # Iteramos sobre los lotes de datos de entrenamiento
    for imagenes, etiquetas in tqdm(train_dataloader):
        # Movemos los datos al dispositivo especificado
        imagenes = imagenes.to(dispositivo)
        etiquetas = etiquetas.to(dispositivo)
        
        # Reiniciamos los gradientes del optimizador
        optimizador.zero_grad()
        
        # Hacemos una predicción con el modelo
        predicciones = modelo(imagenes) 
        
        # Calculamos la pérdida
        perdida = funciondeperdida(predicciones, etiquetas )
        
        # Calculamos los gradientes y actualizamos los parámetros del modelo
        perdida.backward()
        optimizador.step()
        
        # Calculamos la precisión y el error para el lote actual
        _, predicciones = torch.max(predicciones, 1)
        perdida_train += perdida.item()
        acuracy_train += torch.sum(predicciones == etiquetas).item() / len(etiquetas)
        error += torch.sum(predicciones != etiquetas).item() / len(etiquetas)
        
    # Calculamos la pérdida, precisión y error promedio para el conjunto de entrenamiento
    perdida_train /= len(train_dataloader)
    acuracy_train /= len(train_dataloader )
    error /= len(train_dataloader )
    
    # Almacenamos las métricas de entrenamiento en las listas correspondientes
    perdidas_train.append(perdida_train)
    Acuracy_train.append(acuracy_train)
    error_train.append(error)
    
    # Ponemos el modelo en modo de evaluación
    modelo.eval()
    
    # Inicializamos las variables de pérdida, precisión y error para el conjunto de prueba
    perdida_test = 0
    acuracy_test = 0
    error  = 0
    
    # Iteramos sobre los lotes de datos de prueba
    for imagenes, etiquetas in tqdm(test_dataloader):
        # Movemos los datos al dispositivo especificado
        imagenes = imagenes.to(dispositivo)
        etiquetas = etiquetas.to(dispositivo)
        
        # Desactivamos el cálculo de gradientes para acelerar la inferencia
        with torch.no_grad():
            # Hacemos una predicción con el modelo
            predicciones = modelo(imagenes)
            
            # Calculamos la pérdida
            perdida = funciondeperdida(predicciones, etiquetas )
            perdida_test += perdida.item()
            
            # Calculamos la precisión y el error para el lote actual
            _, predicciones = torch.max(predicciones, 1)
            acuracy_test += torch.sum(predicciones == etiquetas).item() / len(etiquetas)
            error+= torch.sum(predicciones != etiquetas).item() / len(etiquetas)
    
    # Calculamos la pérdida, precisión y error promedio para el conjunto de prueba
    perdida_test /= len(test_dataloader )
    acuracy_test /= len(test_dataloader )
    error /= len(test_dataloader )
    
    # Almacenamos las métricas de prueba en las listas correspondientes
    perdidas_test.append(perdida_test)
    Acuracy_test.append(acuracy_test)
    error_test.append(error)
    
    # Imprimimos las métricas de entrenamiento y prueba para la época actual
    print(f"Epoca {epoca+1}/{epocas}, perdida train: {perdidas_train[-1]}, perdida test: {perdidas_test[-1]}, acuracy train: {Acuracy_train[-1]}, acuracy test: {Acuracy_test[-1]}, error train: {error_train[-1]}, error test: {error_test[-1]}")
```

### Graficas

```python
import matplotlib.pyplot as plt

# Gráfica de pérdidas
plt.figure(figsize=(15, 10))
plt.plot(perdidas_train, label='train')
plt.plot(perdidas_test, label='test')
plt.legend()
plt.title('Pérdidas')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.show()

# Gráfica de precisión
plt.figure(figsize=(15, 10))
plt.plot(Acuracy_train, label='train')
plt.plot(Acuracy_test, label='test')
plt.legend()
plt.title('Precisión')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.show()

# Gráfica de error
plt.figure(figsize=(15, 10))
plt.plot(error_train, label='train')
plt.plot(error_test, label='test')
plt.legend()
plt.title('Error')
plt.xlabel('Época')
plt.ylabel('Error')
plt.show()


```

### Predicciones

```python
# Predecir desde una URL de imagen aleatoria
import requests
from PIL import Image
import random

# Lista de URLs de imágenes
urls = [
    "https://whatnowhou.com/wp-content/uploads/sites/17/2022/08/Crown-Pizza-to-Open-in-Katy-Photo-1.jpg",
    "https://www.gardengourmet.com/sites/default/files/recipes/83393badc124840730cb91e3cd839b93_220516_gg_q3_recipe_hotdogs_v3.jpg",
    "https://img.sndimg.com/food/image/upload/q_92,fl_progressive,w_1200,c_scale/v1/img/recipes/91/82/7/pic74ikYM.jpg",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRm80awe9utbx2ShDIu-PxcArRkKm2BCq4vWA&usqp=CAU",
    "https://uploads-ssl.webflow.com/5e9ebc3fff165933f19fbdbe/61b31c9d289e22335b6753b2_Ice%20Cream%202.jpg",
    "https://food.fnr.sndimg.com/content/dam/images/food/fullset/2015/5/14/2/YW0611H_Taco-Pizza_s4x3.jpg.rend.hgtvcom.616.462.suffix/1433264087767.jpeg",
    "https://www.favfamilyrecipes.com/wp-content/uploads/2022/08/Taco-Pizza-toppings.jpg",
    "https://hoy.com.do/wp-content/uploads/2022/07/ROM_5409.jpg",
    "https://previews.123rf.com/images/igormalovic/igormalovic1604/igormalovic160400468/55349527-ni%C3%B1a-en-la-ilustraci%C3%B3n-de-dibujos-animados-vestido-de-color-rosa.jpg",
    "https://m.media-amazon.com/images/I/61DtAEkJBEL._AC_UF894,1000_QL80_.jpg"
]

# Elegir una URL aleatoria de la lista
url = random.choice(urls)

# Descargar la imagen de la URL y abrirla con la clase Image de PIL
imagen = Image.open(requests.get(url, stream=True).raw)

# Aplicar la transformación de prueba a la imagen y convertirla en un tensor
imag = test_transform(imagen).unsqueeze(0)

# Hacer una predicción con el modelo utilizando la imagen transformada
with torch.no_grad():
    pred = modelo(imag.to(dispositivo))
    result = torch.softmax(pred, dim=1)[0]
    indice = torch.argmax(result)
    result = result[indice] * 100
    
    # Mostrar la imagen y la clasificación resultante
    imagen.resize((192, 192)).show()
    print(f"{'pizza' if indice==1 else 'not pizza'} - {result:.2f}%")

```


## Neuronas

Las neuronas son la unidad básica de procesamiento de la información en el cerebro. Una neurona recibe señales de entrada a través de las dendritas y las procesa. Si la señal de entrada es lo suficientemente fuerte, la neurona dispara una señal de salida a lo largo del axón.

### Estructura de una neurona

<div class="imagen">
<img src="https://miro.medium.com/v2/resize:fit:640/0*yV-rEh63sjNPIfrR" alt="Mcpits"  >
</div>
 

- **Dendritas:** Las dendritas son estructuras en forma de árbol que reciben información a través de conexiones sinápticas. Esta información puede ser sensorial o “computacional” y proviene de otras células nerviosas. Una sola célula puede tener hasta 100,000 entradas, cada una de una célula diferente.
- **Cuerpo celular:** El cuerpo celular o soma contiene el núcleo de la célula y procesa las señales de entrada de las dendritas.
- **Axón:** El axón es una fibra larga que transmite señales de salida a otras neuronas a través de ramas llamadas terminales axónicas.
- **Terminales axónicas:** Las terminales axónicas son ramas del axón que se ramifican en forma de árbol y transmiten señales de salida a otras neuronas a través de conexiones sinápticas.
- **Sinapsis:** Las sinapsis son conexiones entre terminales axónicas y dendritas de otras neuronas. Las señales de salida de una neurona se transmiten a las dendritas de otras neuronas a través de las sinapsis.

## Neuronas artificiales

Las neuronas artificiales son modelos matemáticos o computacionales que se inspiran en las neuronas biológicas, pero que se simplifican enormemente. Las neuronas artificiales son la unidad básica de procesamiento de las redes neuronales artificiales.

### Estructura de una neurona artificial

<div class="imagen">
<img src="https://jontysinai.github.io/assets/article_images/2017-11-11-the-perceptron/bio-vs-MCP.png" alt="Mcpits"  >
</div>
 

- Entradas  : Cada neurona recibe una serie de entradas que representan la información que ingresa al sistema. Estas entradas pueden ser características o variables relevantes para el problema que se está abordando.

- Pesos : Cada entrada tiene asociado un peso que indica su importancia relativa en la neurona. Los pesos pueden ser considerados como los "coeficientes" que ponderan la influencia de cada entrada en la salida de la neurona. Los pesos pueden ajustarse durante el proceso de entrenamiento de la red neuronal para lograr un mejor rendimiento.

- Sesgo : El sesgo es un parámetro adicional que se utiliza para ajustar la salida de la neurona. El sesgo es similar a un peso, pero no está asociado a ninguna entrada en particular. 

- Función de activación : Después de aplicar una combinación lineal de las entradas ponderadas por los pesos, se aplica una función de activación no lineal. Esta función introduce no linealidades en la neurona, lo que le permite capturar relaciones más complejas y realizar una transformación no lineal de los datos de entrada.  

- La salida  de la neurona se calcula aplicando la función de activación a la suma ponderada de las entradas. Dependiendo del tipo de neurona o del contexto de la red neuronal, la salida de la neurona puede ser utilizada como entrada para otras neuronas o como salida final de la red.

#### Funciones de activación

##### Función Sigmoide (Sigmoid)

<div class="imagen">
<img src="https://hvidberrrg.github.io/deep_learning/activation_functions/assets/sigmoid_function.png" alt="Mcpits"  >
</div>
 

Esta función toma un valor real como entrada y devuelve una probabilidad que siempre está entre 0 y 1. La fórmula es f(x) = 1 / (1 + exp(-x)). Su rango de salida es [0, 1]. Una de las características de esta función es que presenta saturación de gradientes para valores extremadamente grandes o pequeños.

##### Función ReLU (Rectified Linear Unit)

<div class="imagen">
<img src="https://images.deepai.org/glossary-terms/rectified-linear-units-1149176.jpg" alt="Mcpits"  >
</div>
 

Esta función devuelve el máximo entre 0 y el valor de entrada. La fórmula es f(x) = max(0, x). Su rango de salida es [0, +inf]. Una de las características de esta función es que no presenta saturación de gradientes en la región positiva.

##### Función Leaky ReLU
 
<div class="imagen">
<img src="https://pic4.zhimg.com/80/v2-b30581f4f198f620340d981ae7c4689b_1440w.webp" alt="Mcpits"  >
</div>

Esta función es similar a la función ReLU, pero permite un pequeño valor negativo cuando la entrada es negativa. La fórmula es f(x) = max(ax, x), donde ‘a’ es un valor pequeño (por ejemplo, 0.01). Su rango de salida es (-inf, +inf). Una de las características de esta función es que resuelve el problema de la “neurona muerta” en ReLU.

##### Función Tanh (Tangente hiperbólica)

<div class="imagen">
<img src="https://static.packt-cdn.com/products/9781838646301/graphics/assets/baaa8e5a-1d75-47f4-aef9-120f57f7c78c.png" alt="Mcpits"  >
</div> 

Esta función toma un valor real como entrada y devuelve un valor entre -1 y 1. La fórmula es f(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x)). Su rango de salida es [-1, 1]. Una de las características de esta función es que es similar a la función sigmoide pero con un rango de salida centrado en 0.

##### Función Identidad (Linear)

<div class="imagen">
<img src="https://d1u2r2pnzqmal.cloudfront.net/content_images/images/113/normal/functions-and-realations-rfunction-two.jpg?1503310286" alt="Mcpits"  >
</div> 
 

Esta función devuelve el valor de entrada sin cambios. La fórmula es f(x) = x. Su rango de salida es (-inf, +inf). Una de las características de esta función es que se utiliza en modelos lineales y regresiones para obtener salidas continuas.

## Redes neuronales artificiales
 
<div class="imagen">
<img src="https://miro.medium.com/v2/resize:fit:640/0*CXuqagS3-m4bHQss" alt="Mcpits"  >
</div> 

Las redes neuronales artificiales (ANN) son un modelo computacional que trata de emular el cerebro humano mediante una combinación de ciencia informática y estadística para resolver problemas comunes en el campo de la inteligencia artificial (IA). Están formadas por capas de nodos, que contienen una capa de entrada, una o varias capas ocultas y una capa de salida. Cada nodo, o neurona artificial, se conecta a otro y tiene un peso y un umbral asociados. Si la salida de un nodo individual está por encima del valor de umbral especificado, dicho nodo se activa y envía datos a la siguiente capa de la red.

[Play Ground](https://playground.tensorflow.org/)

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/ejemploderetneuronalperroygatos.png?raw=true" alt="Mcpits"  >
</div> 

### Arquitecturas

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/arquitecturas.png?raw=true" alt="Mcpits"  >
</div> 

La arquitectura de una red neuronal es la estructura y organización de sus componentes, es decir, cómo están dispuestas las capas y conexiones entre ellas.

La arquitectura está determinada por:

- Capas: Una red neuronal está compuesta por una o más capas de neuronas. Estas capas son unidades funcionales que procesan la información. Se organizan de forma secuencial, y la información fluye en una dirección desde la capa de entrada hasta la capa de salida.

- Neuronas: Las neuronas son las unidades básicas de procesamiento en una red neuronal. Cada neurona toma una entrada, realiza una operación matemática y produce una salida. Las salidas de las neuronas se envían como entradas a las neuronas de la siguiente capa.



- Conexiones: Las conexiones son los enlaces que conectan las neuronas entre diferentes capas. Cada conexión tiene un peso asociado que determina la importancia de la información que se transmite de una neurona a otra.

- Funciones de Activación: Cada neurona tiene asociada una función de activación que introduce no linealidades en el modelo. Al aplicar una función de activación a la salida de una neurona, se introduce la capacidad de aprender relaciones no lineales en los datos.

- Tamaño de la Red: El número de neuronas y capas en la red determina su tamaño. Las redes más grandes, con más capas y neuronas, pueden aprender representaciones más complejas y resolver problemas más complejos, pero también pueden requerir más datos y recursos computacionales.

#### Perceptrón
 
<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/perceptronderosenblank.png?raw=true" alt="Mcpits"  >
</div> 


El perceptrón, desarrollado por Frank Rosenblatt en 1957, representa la forma más elemental de una red neuronal artificial, consistiendo en una única neurona. El perceptrón es un clasificador binario que toma un vector de entrada y produce una salida binaria, esto se debe a su función de activación, la cual es una función escalón.

#### Redes Neuronales Feedforward

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/redneuronalfeedforward.png?raw=true" alt="Mcpits"  >
</div> 


Las Redes Neuronales Feedforward son un tipo de arquitectura que consta de una capa de entrada, una capa de salida y una o más capas ocultas. Aunque también se les conoce como perceptrones multicapa (MLP), es esencial destacar que están formadas por neuronas sigmoideas en lugar de perceptrones, lo que les permite un mejor procesamiento de problemas no lineales. Estas redes neuronales feedforward tienen diversas aplicaciones, incluyendo la visión artificial, el procesamiento del lenguaje natural y otras tareas similares. Gracias a su capacidad para aprender y representar relaciones no lineales en los datos, se han convertido en una poderosa herramienta en el campo de la inteligencia artificial.

#### Redes Neuronales Recurrentes (RNN)
 
<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/redneuronalrecurrente.png?raw=true" alt="Mcpits"  >
</div> 

Las Redes Neuronales Recurrentes (RNN) se caracterizan por su estructura de bucles de retroalimentación. Estas redes neuronales son especialmente adecuadas para tareas que implican secuencias temporales u ordenadas, como predecir resultados futuros, como pronósticos del mercado de valores o ventas en cadenas de tiendas. Además de su aplicabilidad en pronósticos, las RNN también han demostrado un excelente rendimiento en tareas como la traducción de idiomas, el procesamiento del lenguaje natural (NLP) y el reconocimiento de voz. Su versatilidad y capacidad para manejar contextos secuenciales las han llevado a formar parte de aplicaciones como Siri y Google Translate, impulsando así la calidad y la eficiencia de estas herramientas de asistencia y traducción. Gracias a sus bucles de retroalimentación, las RNN pueden recordar información pasada y adaptarse a las características cambiantes en datos secuenciales, lo que las convierte en una poderosa opción para tareas temporales y ordenadas.

#### Redes neuronales convolucionales (ConvNets o CNN)
 
<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/convnet.jpeg?raw=true" alt="Mcpits"  >
</div> 

Son un tipo especializado de red neuronal utilizado principalmente para el análisis de imágenes visuales. En lugar de la multiplicación de matrices general, utilizan la operación de convolución en al menos una de sus capas. Están específicamente diseñadas para procesar datos de píxeles y tienen diversas aplicaciones, como el reconocimiento y procesamiento de imágenes en tareas como clasificación, segmentación y análisis médico. Las CNN también se conocen como Redes Neuronales Artificiales Invariantes al Desplazamiento o al Espacio (SIANN) debido a su arquitectura de peso compartido, que proporciona respuestas equivariantes a la traducción, conocidas como mapas de características. En comparación con los perceptrones multicapa, las CNN abordan el sobreajuste aprovechando la jerarquía de patrones en los datos y ensamblan patrones complejos a partir de patrones más simples. Esto permite que la red aprenda representaciones abstractas de la entrada al procesar características en regiones más pequeñas y simples.

#### Variational Autoencoder (VAE)

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/autoencoder_orig.png?raw=true" alt="Mcpits"  >
</div> 
 

Un VAE (Variational Autoencoder) es una arquitectura de red neuronal que pertenece a la familia de modelos gráficos probabilísticos y métodos bayesianos variacionales. Es una extensión del autoencoder tradicional, que busca aprender una representación compacta y útil de los datos en un espacio latente de menor dimensión. Sin embargo, a diferencia del autoencoder, el VAE regulariza la distribución del espacio latente durante el entrenamiento para que siga una distribución específica, generalmente una distribución normal multivariante. Esto permite que el VAE genere nuevos datos al muestrear puntos en el espacio latente y decodificarlos para producir nuevas salidas. En resumen, el VAE es una poderosa herramienta para la generación de datos a partir de una distribución latente controlada y ha encontrado aplicaciones en diversos campos, como la generación de imágenes, el modelado de lenguaje y la música.

#### UNET

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/u-net-architecture-1024x682.png?raw=true" alt="Mcpits"  >
</div> 
 

UNET es una arquitectura de red neuronal desarrollada por Olaf Ronneberger et al. en 2015 para la segmentación de imágenes biomédicas en la Universidad de Friburgo, Alemania. Es ampliamente utilizado en tareas de segmentación semántica debido a su eficacia y su capacidad para aprender con menos muestras de entrenamiento.
 
Una UNET se asemeja a una "U" e incluye cuatro bloques codificadores y cuatro bloques decodificadores conectados a través de un puente. El camino de contracción (codificadores) reduce las dimensiones espaciales a la mitad y aumenta el número de filtros en cada bloque codificador.

UNET se introdujo originalmente en el trabajo "U-Net: Redes convolucionales para la segmentación de imágenes biomédicas" de Ronneberger et al. y consta de un camino de contracción y un camino expansivo, siguiendo la arquitectura típica de una red convolucional. Esta estructura de red ha demostrado ser efectiva para la segmentación precisa de imágenes biomédicas.

#### Transformer

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/transformer.png?raw=true" alt="Mcpits"  >
</div> 

El Transformer es una arquitectura de aprendizaje profundo basada en el mecanismo de atención. Destaca por requerir menos tiempo de entrenamiento en comparación con arquitecturas recurrentes anteriores, como LSTM, y ha sido ampliamente utilizado para entrenar grandes modelos de lenguaje en conjuntos de datos extensos, como Wikipedia y Common Crawl, gracias a su procesamiento paralelo de secuencias de entrada.

El modelo toma tokens de entrada tokenizados y, en cada capa, contextualiza cada token con otros tokens de entrada a través del mecanismo de atención. Aunque el modelo Transformer se introdujo en 2017, el mecanismo de atención fue propuesto previamente en 2014 para la traducción automática.

El Transformer ha encontrado aplicaciones no solo en el procesamiento del lenguaje natural, sino también en la visión por computadora, audio y procesamiento multimodal. Además, ha dado lugar al desarrollo de sistemas pre-entrenados como GPT (transformadores pre-entrenados generativos) y BERT (Representaciones del codificador bidireccional a partir de transformadores). Su versatilidad y eficiencia han revolucionado el campo del aprendizaje profundo y han permitido avances significativos en diversas tareas de procesamiento de datos complejas.

### Deep Learning

<div class="imagen">
<img src="https://github.com/YoelPilier/AIASTS/blob/Kirino/neural/redneuronalvsredneuronalprofunda.png?raw=true" alt="Mcpits"  >
</div> 

Deep Learning (Aprendizaje Profundo) es una tecnica de Machine Learning Inteligencia Artificial que se centra en el entrenamiento de redes neuronales artificiales profundas.El Deep Learning es capaz de aprender automáticamente representaciones de alto nivel y abstracciones a partir de los datos de entrada. Es impulsado por grandes cantidades de datos y el poder computacional necesario para entrenar modelos complejos.


### Entrenamiento de una red neuronal

#### pytorch

PyTorch es una biblioteca de aprendizaje automático de código abierto para Python, utilizada principalmente para el procesamiento del lenguaje natural. Fue desarrollado por los equipos de inteligencia artificial de Facebook Inc. en 2016.  

##### Librerías
 
  ```python
!pip install safetensors torch


  import torch
  import torch.nn as nn
  from safetensors.torch  import load_model,save_model

  # Comprobamos si CUDA (tarjeta gráfica) está disponible en el sistema.
  # Si CUDA está disponible, asignamos el dispositivo "cuda" a la variable 'device',
  # de lo contrario, asignamos el dispositivo "cpu".
  device = "cuda" if torch.cuda.is_available() else "cpu"

  ```

  Con PyTorch podemos utilizar tanto la CPU como el GPU. Sin embargo, al elegir uno, debemos configurar todo el código para que utilice el mismo dispositivo. Para lograr esto, utilizaremos la variable device.

##### Definir los datos de entrada y salida para el modelo

```python

x = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float32)

y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float32)
```
  
Para utilizar datos en PyTorch, ya sea para entrenamiento o inferencia, es necesario que los datos estén almacenados en tensores,un tensor es una estructura de datos similar a un arreglo multidimensional que puede contener elementos de varios tipos, pero para un correcto funcionamiento del modelo en PyTorch, se requiere que todos los tensores que representan los datos de entrada y salida sean del mismo tipo, y en este caso, deben ser del tipo torch.float32. Esto garantiza la consistencia en los cálculos y evita errores durante el proceso de entrenamiento e inferencia. Además, los tensores también permiten el cómputo acelerado en GPU si se tiene disponible una tarjeta gráfica compatible con CUDA.

##### Definir el modelo

```python
class RedNeuronal(nn.Module):
  def __init__(self ):
    # Inicializamos la clase padre
    super(RedNeuronal, self).__init__()
    # Definimos la arquitectura de la red neuronal
    self.capa1=nn.Linear(2,2)
    self.capa2=nn.Linear(2,1)
    self.sigmoid=nn.Sigmoid()
    self.relu=nn.ReLU()
      
    # Definimos el método para la propagación hacia adelante
  def forward(self, x):
    x=self.capa1(x)
    x=self.relu(x)
    x=self.capa2(x)
    x=self.sigmoid(x)
    return x
```

Para definir una red neuronal, debemos crear una clase que herede de la clase `nn.Module`. En esta clase, definimos las capas que compondrán el modelo, las funciones de activación y la función `forward`, que define cómo viajará la información a través del modelo. Esta función `forward` se utilizará tanto para el entrenamiento como para la inferencia cuando el modelo sea puesto en producción. 

##### Crear el modelo

```python
# Creamos una instancia de la clase RedNeuronal
modelo = RedNeuronal()
# Enviamos el modelo al dispositivo seleccionado
modelo.to(device)
print(modelo)
```

Para crear una instancia de la clase RedNeuronal, simplemente llamamos a la clase y asignamos la instancia a la variable model. Luego, enviamos el modelo al dispositivo seleccionado, ya sea CPU o GPU, utilizando el método to().

##### Definir hyperparámetros

```python
# Definimos el número de épocas
epocas=50
# Definimos la tasa de aprendizaje
tasa_aprendizaje=3e-2
  
```

Los hiperparámetros son parámetros externos que se utilizan para configurar un algoritmo de aprendizaje automático o una red neuronal antes del proceso de entrenamiento. A diferencia de los parámetros del modelo, que son ajustados internamente durante el proceso de entrenamiento para adaptarse a los datos, los hiperparámetros se establecen antes del entrenamiento y no cambian durante el proceso de aprendizaje. En este caso tenemos el numero de epocas que es la cantidad de veces que el modelo va a ver los datos de entrenamiento, y la tasa de aprendizaje que es la cantidad de ajuste que se le va a dar a los pesos en cada iteración.


##### Definir la función de pérdida y el optimizador

```python
# Definimos la función de pérdida
funcion_perdida=nn.MSELoss()
# Definimos el optimizador
optimizador=torch.optim.Adam(modelo.parameters(), lr=tasa_aprendizaje)

# Definimos las listas para almacenar las métricas de entrenamiento

perdidas_train=[]

Acuracy_train=[]

error_train=[]
```

La función de pérdida es una medida que nos indica la discrepancia entre las predicciones del modelo y los valores reales de los datos de entrenamiento. El objetivo del optimizador es ajustar los pesos del modelo durante el proceso de entrenamiento para minimizar esta función de pérdida.


##### Entrenar el modelo

```python
#Definimos el ciclo de entrenamiento
# Iteramos sobre el número de épocas

for epoca in range(epocas):
  
  # Reiniciamos los gradientes
  optimizador.zero_grad()
  # Hacemos la propagación hacia adelante
  
  x=x.to(device)
  y=y.to(device)
  y_pred=modelo(x)
  # Calculamos la pérdida
  perdida=funcion_perdida(y_pred, y)
  # Hacemos la propagación hacia atrás
  perdida.backward()
  # Actualizamos los pesos
  optimizador.step()
  
  # Guardamos la pérdida
  
  perdidas_train.append(perdida.item())
  
  # Calculamos el error
  
  error_train.append(torch.mean(torch.abs(y_pred-y)).item())
  
  # Calculamos el acuracy
  
  Acuracy_train.append(torch.mean((y_pred.round()==y).float()).item())
  
  # Imprimimos los resultados cada 10 épocas
  
  if (epoca+1)%10==0:
    
    print(f'Época: {epoca+1} - Pérdida: {perdida.item()} - Error: {error_train[-1]} - Acuracy: {Acuracy_train[-1]}')
  
```

El proceso de entrenamiento de una red neuronal consiste en iterar sobre los datos de entrenamiento.En cada iteración, se realizan los siguientes pasos:

- Se reinician los gradientes del optimizador para evitar acumulación en el cálculo del gradiente.
- Se realiza la propagación hacia adelante pasando los datos de entrada (x) por el modelo para obtener las predicciones (y_pred).
- Se calcula la pérdida (diferencia entre las predicciones y los valores reales, y) utilizando la función de pérdida definida anteriormente (funcion_perdida).
- Se realiza la propagación hacia atrás (backpropagation) para calcular los gradientes de los pesos del modelo con respecto a la pérdida.
- Se actualizan los pesos del modelo utilizando el optimizador (optimizador.step()) para minimizar la pérdida en la siguiente iteración.

##### Graficas

```python
from matplotlib import pyplot as plt

# Graficamos la pérdida, el error y la precisión durante el entrenamiento
fig, axs = plt.subplots(3, 1, figsize=(8, 12))

axs[0].plot(perdidas_train)
axs[0].set_title("Pérdida durante el entrenamiento")
axs[0].set_xlabel("Época")
axs[0].set_ylabel("Pérdida")

axs[1].plot(error_train)
axs[1].set_title("Error durante el entrenamiento")
axs[1].set_xlabel("Época")
axs[1].set_ylabel("Error")

axs[2].plot(Acuracy_train)
axs[2].set_title("Precisión durante el entrenamiento")
axs[2].set_xlabel("Época")
axs[2].set_ylabel("Precisión")

plt.tight_layout()
plt.show()

```

##### Evaluar el modelo

  ```python 
    with torch.no_grad():
    y_pred=modelo(x)
    y_pred_clase=y_pred.round()
    print(y_pred_clase)
  ```
  
  Para evaluar el modelo, simplemente pasamos los datos de entrada (x) por el modelo para obtener las predicciones (y_pred). Luego, redondeamos las predicciones (y_pred_clase) para obtener la clase predicha (0 o 1) y la imprimimos.

##### Guardar el modelo

```python
    # Guardamos el modelo
save_model(modelo, "modelo.safetensors")

```

##### Cargar el modelo

```python
    # Cargamos el modelo 
modelo=RedNeuronal()
load_model(modelo, "modelo.safetensors")
modelo.eval()

```

##### Inferencia

```python
def XOR(evaluar):
    
    evaluar=torch.tensor([evaluar],dtype=torch.float32)
    evaluar=modelo(evaluar)
    evaluar=evaluar.item()
    return evaluar

print(XOR([1,0]))
```

#### Otros problemas de clasificación


- AND
- OR
- XOR
- NOT
- XNOR
- NOR
- NAND



 